# Guided Project1: Profitable App Profiles for the App Store and Google Play Markets

Our company only build apps that are free to download and install, and that are directed toward an English-speaking audience. This means that we'll need to:

*Remove non-English apps like 爱奇艺PPS -《欢乐颂2》电视剧热播.

*Remove apps that aren't free.

Our goal for this project is to analyze data to help our developers understand what type of apps are likely to attract more users.

## Step1: Converting .csv files to list-of-lists

In [1]:
def list_of_list(dataset):
    opened_file = open(dataset,encoding='utf8')
    from csv import reader
    read_file = reader(opened_file)
    return list(read_file)

In [2]:
appg_data = list_of_list("googleplaystore.csv")
apps_data = list_of_list("AppleStore.csv")

## Step2: Printing a part of the list-of-lists to visualize the dataset

In [3]:
def explore_data(dataset, start, end, rows_and_columns=True):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line after each row

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

In [4]:
explore_data(appg_data[1:],0,2)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', '7-Jan-18', '1.0.0', '4.0.3 and up', '', '', '', '', '']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', '15-Jan-18', '2.0.0', '4.0.3 and up', '', '', '', '', '']


Number of rows: 10841
Number of columns: 18


In [5]:
explore_data(apps_data[1:],0,2)

['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


Number of rows: 7197
Number of columns: 16


## Step3: Data Cleaning

We need to:

* Detect inaccurate data, and correct or remove it.

* Detect duplicate data, and remove the duplicates.

* Modify data to fit to our purpose (remove non-English and paid apps)

### Step 3a: Detecting inaccurate data (not all column entries are present in a row) and removing them

In [6]:
# Detecting inaccurate data (missing entry in a row)

def data_issue(listoflist):
    header_length = len(listoflist[0])
    for row in listoflist:
        row_length = len(row)
        if row_length != header_length:
            print(row)
            print(listoflist.index(row))

In [7]:
appg_issues = data_issue(appg_data[1:])
print('\n')
apps_issues = data_issue(apps_data[1:])

In [8]:
del appg_data[10472]

In [9]:
len(appg_data)

10841

### Step 3b: Detecting duplicate data and keeping the ones with the highest reviews

In [10]:
# Detecting Duplicate data in AppleStore (duplicate app names)

def duplicate_Apple_app(dataset):
    unique_app = []
    not_unique_app = []
    for row in dataset:
        if row[1] not in unique_app:
            unique_app.append(row[1])
        else:
            not_unique_app.append(row[1])
            
    print(not_unique_app)
    print('\n')
    print(len(not_unique_app))

In [11]:
duplicate_Apple_app(apps_data)

['Mannequin Challenge', 'VR Roller Coaster']


2


In [12]:
# Detecting duplicate data in Google Store (duplicate app names)

def duplicate_Google_app(dataset):
    unique_app = []
    not_unique_app = []
    for row in dataset:
        if row[0] not in unique_app:
            unique_app.append(row[0])
        else:
            not_unique_app.append(row[0])
            
    print(not_unique_app[:5])
    print('\n')
    print(len(not_unique_app))

In [13]:
duplicate_Google_app(appg_data)

['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings']


1181


In [14]:
# Remove duplicate apps from the Google Store. From the apps, which have same name, we will only keep the one with the 
# highest rating

reviews_max = {}

for row in appg_data[1:]:
    name = row[0]
    reviews = float(row[3])
    
    if name not in reviews_max:
        reviews_max[name] = reviews
        
    elif name in reviews_max and (reviews > reviews_max[name]):
        reviews_max[name] = reviews        

Total number of duplicate apps = 1181

So, total number of apps in Google Store - 1181 should be equal to the list of unique apps created above i.e. reviews_max dictionary

In [15]:
print("Expected length: ",len(appg_data[1:])-1181)
print("Actual length: ", len(reviews_max))

Expected length:  9659
Actual length:  9659


Let's remove all the apps from appg_data, which aren't in reviews_max

In [16]:
appg_clean = []
appg_already_added = []

for row in appg_data[1:]:
    name = row[0]
    reviews = float(row[3])
    
    if (reviews_max[name] == reviews) and (name not in appg_already_added):
        appg_clean.append(row)
        appg_already_added.append(name)

In [17]:
explore_data(appg_clean,0,3,True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', '7-Jan-18', '1.0.0', '4.0.3 and up', '', '', '', '', '']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', '1-Aug-18', '1.2.4', '4.0.3 and up', '', '', '', '', '']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', '8-Jun-18', 'Varies with device', '4.2 and up', '', '', '', '', '']


Number of rows: 9659
Number of columns: 18


## Step 3c: Removing Non-English Apps (Use of "ord" function)

1. Write a function that takes in a string and returns False if there's any character in the string that doesn't belong to the set of common English characters, otherwise it returns True.

2. All English apps with up to three emoji or other special characters will still be labeled as English. Our filter function is still not perfect, but it should be fairly effective

In [18]:
def english(app_name):
    count_special_char = 0
    for char in app_name:
        if ord(char) >127:
            count_special_char += 1
            if count_special_char>3:
                return False
                 
    return True          

In [19]:
english('爱奇艺PPS -《欢乐颂2》电视剧热播')

False

In [20]:
apps_english = []
apps_non_english = []

for row in apps_data:
    if english(row[1]):
        apps_english.append(row)
    else:
        apps_non_english.append(row)

In [21]:
explore_data(apps_english,0,3)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


Number of rows: 6184
Number of columns: 16


In [22]:
appg_english = []
appg_non_english = []

for row in appg_clean:
    if english(row[0]):
        appg_english.append(row)
    else:
        appg_non_english.append(row)

In [23]:
explore_data(appg_english,0,3)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', '7-Jan-18', '1.0.0', '4.0.3 and up', '', '', '', '', '']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', '1-Aug-18', '1.2.4', '4.0.3 and up', '', '', '', '', '']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', '8-Jun-18', 'Varies with device', '4.2 and up', '', '', '', '', '']


Number of rows: 9614
Number of columns: 18


## Step 4: Isolate Free Aps

In [24]:
apps_free = []

for row in apps_english[1:]:
    price = float(row[4])
    
    if price == 0:
        apps_free.append(row)

In [25]:
explore_data(apps_free,0,3)

['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


Number of rows: 3222
Number of columns: 16


In [26]:
appg_free = []

for row in appg_english[1:]:
    price = float(row[7])
    
    if price == 0:
        appg_free.append(row)

In [27]:
explore_data(appg_free,0,3)

['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', '1-Aug-18', '1.2.4', '4.0.3 and up', '', '', '', '', '']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', '8-Jun-18', 'Varies with device', '4.2 and up', '', '', '', '', '']


['Pixel Draw - Number Art Coloring Book', 'ART_AND_DESIGN', '4.3', '967', '2.8M', '100,000+', 'Free', '0', 'Everyone', 'Art & Design;Creativity', '20-Jun-18', '1.1', '4.4 and up', '', '', '', '', '']


Number of rows: 8863
Number of columns: 18


## Data Analysis:

As we mentioned in the introduction, our aim is to determine the kinds of apps that are likely to attract more users because our revenue is highly influenced by the number of people using our apps.

To minimize risks and overhead, our validation strategy for an app idea is comprised of three steps:

Build a minimal Android version of the app, and add it to Google Play.
If the app has a good response from users, we develop it further.
If the app is profitable after six months, we build an iOS version of the app and add it to the App Store.
Because our end goal is to add the app on both Google Play and the App Store, we need to find app profiles that are successful on both markets. For instance, a profile that works well for both markets might be a productivity app that makes use of gamification.

Let's begin the analysis by getting a sense of what are the most common genres for each market. For this, we'll need to build frequency tables for a few columns in our data sets.

### Step 1: Frequency table of any column in AppleStore or GoogleplayStore dataset

In [28]:
def freq_table(dataset,index):
    frequency_table = {}
    total = 0
    
    for row in dataset:
        column_value = row[index]
        total += 1
        
        if column_value not in frequency_table:
            frequency_table[column_value] = 1
            
        else:
            frequency_table[column_value] += 1
    
    percentage_table = {}
    
    for key in frequency_table:
        percentage_table[key] = (frequency_table[key]/total)*100
        
    return percentage_table

def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

We start by examining the frequency table for the prime_genre column of the App Store data set.

In [29]:
display_table(apps_free,-5)

Games : 58.16263190564867
Entertainment : 7.883302296710118
Photo & Video : 4.9658597144630665
Education : 3.662321539416512
Social Networking : 3.2898820608317814
Shopping : 2.60707635009311
Utilities : 2.5139664804469275
Sports : 2.1415270018621975
Music : 2.0484171322160147
Health & Fitness : 2.0173805090006205
Productivity : 1.7380509000620732
Lifestyle : 1.5828677839851024
News : 1.3345747982619491
Travel : 1.2414649286157666
Finance : 1.1173184357541899
Weather : 0.8690254500310366
Food & Drink : 0.8069522036002483
Reference : 0.5586592178770949
Business : 0.5276225946617008
Book : 0.4345127250155183
Navigation : 0.186219739292365
Medical : 0.186219739292365
Catalogs : 0.12414649286157665


We can see that among the free English apps, more than a half (58.16%) are games. Entertainment apps are close to 8%, followed by photo and video apps, which are close to 5%. Only 3.66% of the apps are designed for education, followed by social networking apps which amount for 3.29% of the apps in our data set.

The general impression is that App Store (at least the part containing free English apps) is dominated by apps that are designed for fun (games, entertainment, photo and video, social networking, sports, music, etc.), while apps with practical purposes (education, shopping, utilities, productivity, lifestyle, etc.) are more rare. However, the fact that fun apps are the most numerous doesn't also imply that they also have the greatest number of users — the demand might not be the same as the offer.

Let's continue by examining the Genres and Category columns of the Google Play data set (two columns which seem to be related).

In [30]:
display_table(appg_free, 1) # Category

FAMILY : 18.910075595170937
GAME : 9.725826469592688
TOOLS : 8.462146000225657
BUSINESS : 4.592124562789123
LIFESTYLE : 3.9038700214374367
PRODUCTIVITY : 3.8925871601038025
FINANCE : 3.7007785174320205
MEDICAL : 3.5315355974275078
SPORTS : 3.396141261423897
PERSONALIZATION : 3.3058783707548236
COMMUNICATION : 3.2381812027530184
HEALTH_AND_FITNESS : 3.0802211440821394
PHOTOGRAPHY : 2.944826808078529
NEWS_AND_MAGAZINES : 2.798149610741284
SOCIAL : 2.6627552747376737
TRAVEL_AND_LOCAL : 2.335552296062281
SHOPPING : 2.245289405393208
BOOKS_AND_REFERENCE : 2.1437436533904997
DATING : 1.8616721200496444
VIDEO_PLAYERS : 1.7939749520478394
MAPS_AND_NAVIGATION : 1.399074805370642
FOOD_AND_DRINK : 1.241114746699763
EDUCATION : 1.1621347173643235
ENTERTAINMENT : 0.9590432133589079
LIBRARIES_AND_DEMO : 0.9364774906916393
AUTO_AND_VEHICLES : 0.9251946293580051
HOUSE_AND_HOME : 0.8236488773552973
WEATHER : 0.8010831546880289
EVENTS : 0.7108202640189552
PARENTING : 0.6544059573507841
ART_AND_DESIGN : 

The landscape seems significantly different on Google Play: there are not that many apps designed for fun, and it seems that a good number of apps are designed for practical purposes (family, tools, business, lifestyle, productivity, etc.). However, if we investigate this further, we can see that the family category (which accounts for almost 19% of the apps) means mostly games for kids.

Even so, practical apps seem to have a better representation on Google Play compared to App Store. This picture is also confirmed by the frequency table we see for the Genres column:

In [31]:
display_table(appg_free, 9) # Genres

Tools : 8.450863138892023
Entertainment : 6.070179397495204
Education : 5.348076272142616
Business : 4.592124562789123
Productivity : 3.8925871601038025
Lifestyle : 3.8925871601038025
Finance : 3.7007785174320205
Medical : 3.5315355974275078
Sports : 3.463838429425702
Personalization : 3.3058783707548236
Communication : 3.2381812027530184
Action : 3.102786866749408
Health & Fitness : 3.0802211440821394
Photography : 2.944826808078529
News & Magazines : 2.798149610741284
Social : 2.6627552747376737
Travel & Local : 2.324269434728647
Shopping : 2.245289405393208
Books & Reference : 2.1437436533904997
Simulation : 2.042197901387792
Dating : 1.8616721200496444
Arcade : 1.8503892587160102
Video Players & Editors : 1.771409229380571
Casual : 1.7601263680469368
Maps & Navigation : 1.399074805370642
Food & Drink : 1.241114746699763
Puzzle : 1.128286133363421
Racing : 0.9928917973598104
Role Playing : 0.9364774906916393
Libraries & Demo : 0.9364774906916393
Auto & Vehicles : 0.9251946293580051


The difference between the Genres and the Category columns is not crystal clear, but one thing we can notice is that the Genres column is much more granular (it has more categories). We're only looking for the bigger picture at the moment, so we'll only work with the Category column moving forward.

Up to this point, we found that the App Store is dominated by apps designed for fun, while Google Play shows a more balanced landscape of both practical and for-fun apps. Now we'd like to get an idea about the kind of apps that have most users.

## Most Popular Apps by Genre on the App Store
One way to find out what genres are the most popular (have the most users) is to calculate the average number of installs for each app genre. For the Google Play data set, we can find this information in the Installs column, but for the App Store data set this information is missing. As a workaround, we'll take the total number of user ratings as a proxy, which we can find in the rating_count_tot app.

Below, we calculate the average number of user ratings per app genre on the App Store:

In [32]:
def freq2_table(dataset,index):
    frequency_table = {}
    total = 0
    
    for row in dataset:
        column_value = row[index]
        total += 1
        
        if column_value not in frequency_table:
            frequency_table[column_value] = 1
            
        else:
            frequency_table[column_value] += 1
            
    return frequency_table




apps_frequency_dict = freq2_table(apps_free,11)
apps_genre_avg_rating = {}
    
for genre in apps_frequency_dict:
    genre_name = genre
    genre_app_count = apps_frequency_dict[genre]
    genre_rating_sum = 0
        
    for app in apps_free:
        if genre_name == app[11]:
            genre_rating_sum += float(app[5])    
            
    average_rating = genre_rating_sum/genre_app_count
                
    apps_genre_avg_rating[genre_name] = average_rating
    
    print(genre_name, " : ",average_rating)

Social Networking  :  71548.34905660378
Photo & Video  :  28441.54375
Games  :  22788.6696905016
Music  :  57326.530303030304
Reference  :  74942.11111111111
Health & Fitness  :  23298.015384615384
Weather  :  52279.892857142855
Utilities  :  18684.456790123455
Travel  :  28243.8
Shopping  :  26919.690476190477
News  :  21248.023255813954
Navigation  :  86090.33333333333
Lifestyle  :  16485.764705882353
Entertainment  :  14029.830708661417
Food & Drink  :  33333.92307692308
Sports  :  23008.898550724636
Book  :  39758.5
Finance  :  31467.944444444445
Education  :  7003.983050847458
Productivity  :  21028.410714285714
Business  :  7491.117647058823
Catalogs  :  4004.0
Medical  :  612.0


On average, navigation apps have the highest number of user reviews, but this figure is heavily influenced by Waze and Google Maps, which have close to half a million user reviews together:

In [33]:
for app in apps_free:
    if app[-5] == 'Navigation':
        print(app[1], ':', app[5]) # print name and number of ratings

Waze - GPS Navigation, Maps & Real-time Traffic : 345046
Google Maps - Navigation & Transit : 154911
Geocaching® : 12811
CoPilot GPS – Car Navigation & Offline Maps : 3582
ImmobilienScout24: Real Estate Search in Germany : 187
Railway Route Search : 5


The same pattern applies to social networking apps, where the average number is heavily influenced by a few giants like Facebook, Pinterest, Skype, etc. Same applies to music apps, where a few big players like Pandora, Spotify, and Shazam heavily influence the average number.

Our aim is to find popular genres, but navigation, social networking or music apps might seem more popular than they really are. The average number of ratings seem to be skewed by very few apps which have hundreds of thousands of user ratings, while the other apps may struggle to get past the 10,000 threshold. We could get a better picture by removing these extremely popular apps for each genre and then rework the averages, but we'll leave this level of detail for later.

Reference apps have 74,942 user ratings on average, but it's actually the Bible and Dictionary.com which skew up the average rating:

In [34]:
for app in apps_free:
    if app[-5] == 'Reference':
        print(app[1], ':', app[5])

Bible : 985920
Dictionary.com Dictionary & Thesaurus : 200047
Dictionary.com Dictionary & Thesaurus for iPad : 54175
Google Translate : 26786
Muslim Pro: Ramadan 2017 Prayer Times, Azan, Quran : 18418
New Furniture Mods - Pocket Wiki & Game Tools for Minecraft PC Edition : 17588
Merriam-Webster Dictionary : 16849
Night Sky : 12122
City Maps for Minecraft PE - The Best Maps for Minecraft Pocket Edition (MCPE) : 8535
LUCKY BLOCK MOD ™ for Minecraft PC Edition - The Best Pocket Wiki & Mods Installer Tools : 4693
GUNS MODS for Minecraft PC Edition - Mods Tools : 1497
Guides for Pokémon GO - Pokemon GO News and Cheats : 826
WWDC : 762
Horror Maps for Minecraft PE - Download The Scariest Maps for Minecraft Pocket Edition (MCPE) Free : 718
VPN Express : 14
Real Bike Traffic Rider Virtual Reality Glasses : 8
教えて!goo : 0
Jishokun-Japanese English Dictionary & Translator : 0


However, this niche seems to show some potential. One thing we could do is take another popular book and turn it into an app where we could add different features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes about the book, etc. On top of that, we could also embed a dictionary within the app, so users don't need to exit our app to look up words in an external app.

This idea seems to fit well with the fact that the App Store is dominated by for-fun apps. This suggests the market might be a bit saturated with for-fun apps, which means a practical app might have more of a chance to stand out among the huge number of apps on the App Store.

Other genres that seem popular include weather, book, food and drink, or finance. The book genre seem to overlap a bit with the app idea we described above, but the other genres don't seem too interesting to us:

Weather apps — people generally don't spend too much time in-app, and the chances of making profit from in-app adds are low. Also, getting reliable live weather data may require us to connect our apps to non-free APIs.

Food and drink — examples here include Starbucks, Dunkin' Donuts, McDonald's, etc. So making a popular food and drink app requires actual cooking and a delivery service, which is outside the scope of our company.

Finance apps — these apps involve banking, paying bills, money transfer, etc. Building a finance app requires domain knowledge, and we don't want to hire a finance expert just to build an app.

Now let's analyze the Google Play market a bit.

## Most Popular Apps by Genre on Google Play
For the Google Play market, we actually have data about the number of installs, so we should be able to get a clearer picture about genre popularity. However, the install numbers don't seem precise enough — we can see that most values are open-ended (100+, 1,000+, 5,000+, etc.).

One problem with this data is that is not precise. For instance, we don't know whether an app with 100,000+ installs has 100,000 installs, 200,000, or 350,000. However, we don't need very precise data for our purposes — we only want to get an idea which app genres attract the most users, and we don't need perfect precision with respect to the number of users.

We're going to leave the numbers as they are, which means that we'll consider that an app with 100,000+ installs has 100,000 installs, and an app with 1,000,000+ installs has 1,000,000 installs, and so on.

To perform computations, however, we'll need to convert each install number to float — this means that we need to remove the commas and the plus characters, otherwise the conversion will fail and raise an error. We'll do this directly in the loop below, where we also compute the average number of installs for each genre (category).

In [36]:
def freq3_table(dataset,index):
    frequency_table = {}
    total = 0
    
    for row in dataset:
        column_value = row[index]
        total += 1
        
        if column_value not in frequency_table:
            frequency_table[column_value] = 1
            
        else:
            frequency_table[column_value] += 1
            
    return frequency_table




appg_frequency_dict = freq3_table(appg_free,1)
appg_genre_avg_rating = {}
    
for genre in appg_frequency_dict:
    genre_name = genre
    genre_app_count = appg_frequency_dict[genre]
    genre_install_sum = 0
        
    for app in appg_free:
        if genre_name == app[1]:
            genre_install = app[5].replace("+","")
            genre_install = genre_install.replace(",","")
            genre_install_sum += float(genre_install)    
            
    average_rating = genre_install_sum/genre_app_count
                
    appg_genre_avg_rating[genre_name] = average_rating
    
    print(genre_name, " : ",average_rating)

ART_AND_DESIGN  :  2021626.7857142857
AUTO_AND_VEHICLES  :  647317.8170731707
BEAUTY  :  513151.88679245283
BOOKS_AND_REFERENCE  :  8767811.894736841
BUSINESS  :  1712290.1474201474
COMICS  :  817657.2727272727
COMMUNICATION  :  38456119.167247385
DATING  :  854028.8303030303
EDUCATION  :  1833495.145631068
ENTERTAINMENT  :  11640705.88235294
EVENTS  :  253542.22222222222
FINANCE  :  1387692.475609756
FOOD_AND_DRINK  :  1924897.7363636363
HEALTH_AND_FITNESS  :  4188821.9853479853
HOUSE_AND_HOME  :  1331540.5616438356
LIBRARIES_AND_DEMO  :  638503.734939759
LIFESTYLE  :  1437816.2687861272
GAME  :  15588015.603248259
FAMILY  :  3695641.8198090694
MEDICAL  :  120550.61980830671
SOCIAL  :  23253652.127118643
SHOPPING  :  7036877.311557789
PHOTOGRAPHY  :  17840110.40229885
SPORTS  :  3638640.1428571427
TRAVEL_AND_LOCAL  :  13984077.710144928
TOOLS  :  10801391.298666667
PERSONALIZATION  :  5218893.815699658
PRODUCTIVITY  :  16787331.344927534
PARENTING  :  542603.6206896552
WEATHER  :  507